1. Install requirements

In [1]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


2. Dataset defined

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'open a file':0,
      'ocr a file':1,
      'open any RSS feed':2,
      'search a book':3,
      'tts a file':4
      }

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['Category']]
        self.texts = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
                for text in df['Text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y


import pandas as pd
text_df = pd.read_csv('./datasets.csv')
df = pd.DataFrame(text_df)

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])
print(len(df_train),len(df_val), len(df_test))


118 15 15


d:\Environment\Anacode\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


3. Modelling

In [2]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [4]:
import torch
print (torch.cuda.is_available())

True


 4. Training Models

In [15]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):
    train, val = Dataset(train_data), Dataset(val_data)
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if torch.cuda.is_available():
            model = model.cuda()
            criterion = criterion.cuda()
    for epoch_num in range(epochs):
            total_acc_train = 0
            total_loss_train = 0
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0
            with torch.no_grad():
                for val_input, val_label in val_dataloader:
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'''Epochs: {epoch_num + 1}
              | Train Loss: {total_loss_train / len(train_data): .3f}
              | Train Accuracy: {total_acc_train / len(train_data): .3f}
              | Val Loss: {total_loss_val / len(val_data): .3f}
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')

In [16]:
EPOCHS = 60
model = BertClassifier()
LR = 1e-6
train(model, df_train, df_val, LR, EPOCHS)
# Save models
torch.save(model, 'classifier.pth')

100%|██████████| 59/59 [00:07<00:00,  7.38it/s]


Epochs: 1
              | Train Loss:  0.828
              | Train Accuracy:  0.178
              | Val Loss:  0.895
              | Val Accuracy:  0.067


100%|██████████| 59/59 [00:07<00:00,  7.56it/s]


Epochs: 2
              | Train Loss:  0.790
              | Train Accuracy:  0.297
              | Val Loss:  0.887
              | Val Accuracy:  0.200


100%|██████████| 59/59 [00:07<00:00,  7.55it/s]


...

Epochs: 58
              | Train Loss:  0.024
              | Train Accuracy:  1.000
              | Val Loss:  0.040
              | Val Accuracy:  1.000


100%|██████████| 59/59 [00:07<00:00,  7.67it/s]


Epochs: 59
              | Train Loss:  0.024
              | Train Accuracy:  1.000
              | Val Loss:  0.047
              | Val Accuracy:  1.000


100%|██████████| 59/59 [00:07<00:00,  7.67it/s]


Epochs: 60
              | Train Loss:  0.020
              | Train Accuracy:  1.000
              | Val Loss:  0.042
              | Val Accuracy:  1.000


5. Evaluation

In [17]:
def evaluate(model, test_data):

    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

Test Accuracy:  1.000


6. Predict

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

labels = {'open a file': 0, 'delete a file': 1, 'ocr a file': 2, 'get latest rss titles': 3, 'search a book': 4, 'tts a file': 5}

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=len(labels))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load('./classifier.pth')
model.to(device)
if torch.cuda.is_available():
        model = model.cuda()
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

model.eval()

def predict(model, sentence, tokenizer, labels, device):
    inputs = tokenizer(sentence, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
    input_id = inputs['input_ids'].to(device)
    mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_id=input_id, mask=mask)

    prediction = torch.argmax(outputs, dim=1).item()

    return prediction

test_sentence = "I want to open the file test.md."
predicted_label = predict(model, test_sentence, tokenizer, labels, device)
print(f"Predicted label for '{test_sentence}': {predicted_label}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted label for 'I want to open the file test.md.': 0
